# Named Entity Recognition for the decicontas.br dataset

In [23]:
import re
import ast
import json
import pprint

import pandas as pd

from typing import List, Tuple, Dict
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import StringPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.metrics import precision_score, recall_score, f1_score
from tools.fewshot import build_fewshot_prompt
from dotenv import load_dotenv

load_dotenv()

gpt4o = AzureChatOpenAI(
    deployment_name="gpt-4o",  # deployment com modelo gpt-4o
    model_name="gpt-4o",       # opcional para compatibilidade
)

gpt35 = AzureChatOpenAI(
    deployment_name="gpt-35",  # outro deployment, com gpt-35-turbo
    model_name="gpt-35-turbo",
)

gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

In [2]:
df_fewshot = pd.read_json("dataset/labeled_data/fewshot_decicontas.json")
fewshot_json = df_fewshot[['annotations', 'data']]

df_decicontas = pd.read_json("dataset/labeled_data/decicontas.json")
decicontas_json = df_decicontas[['annotations', 'data']]


In [3]:
EXEMPLOS = """

Você é um modelo de linguagem treinado para extrair entidades de textos. 
Sua tarefa é identificar e extrair entidades nomeadas de um texto fornecido. 
As entidades podem ter os seguintes labels:

- OBRIGACAO_MULTA: Obrigação imposta sob pena de multa, geralmente com natureza cominatória.
- OBRIGACAO: Obrigação de fazer ou não fazer, sem multa diretamente associada.
- RECOMENDACAO: Sugestões ou recomendações não vinculativas emitidas pelo tribunal.
- MULTA_FIXA: Multa de valor determinado, sem variação percentual.
- MULTA_PERCENTUAL: Multa definida como um percentual sobre determinado valor.
- RESSARCIMENTO: Determinação de devolução de recursos públicos ao erário.

Sempre retorne no seguinte formato, e apenas em json como um array, não escreva "Entidades extraídas", e apenas os labels apresentados:
"""

for _, row in df_fewshot.iterrows():
    texto = row['data']['text']
    entidades = json.dumps([x['value'] for x in row['annotations'][0]['result']])
    EXEMPLOS += f"Texto: {texto}\nEntidades extraídas: {entidades}\n"
EXEMPLOS += "Texto: {input}\nEntidades extraídas: \n"



In [4]:
print(EXEMPLOS)



Você é um modelo de linguagem treinado para extrair entidades de textos. 
Sua tarefa é identificar e extrair entidades nomeadas de um texto fornecido. 
As entidades podem ter os seguintes labels:

- OBRIGACAO_MULTA: Obrigação imposta sob pena de multa, geralmente com natureza cominatória.
- OBRIGACAO: Obrigação de fazer ou não fazer, sem multa diretamente associada.
- RECOMENDACAO: Sugestões ou recomendações não vinculativas emitidas pelo tribunal.
- MULTA_FIXA: Multa de valor determinado, sem variação percentual.
- MULTA_PERCENTUAL: Multa definida como um percentual sobre determinado valor.
- RESSARCIMENTO: Determinação de devolução de recursos públicos ao erário.

Sempre retorne no seguinte formato, e apenas em json como um array, não escreva "Entidades extraídas", e apenas os labels apresentados:
Texto:                                 DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, em consonância com a informação do Corpo Técnico e com o parecer do Ministér

# Testing with golden labels

In [ ]:
# Output parser simples de string
output_parser = StrOutputParser()

# Cadeia como função com str.format()
def run_chain(texto_input: str) -> List[dict]:
    prompt = EXEMPLOS.replace("{input}", texto_input)
    resposta = output_parser.invoke(gpt4o.invoke(prompt)).content.strip()

    # Remove blocos markdown e limpa
    resposta = re.sub(r"```json|```", "", resposta).strip()

    try:
        return ast.literal_eval(resposta)
    except Exception as e:
        print(f"[Erro parseando resposta]:\n{resposta}\n-> {e}")
        return []

In [7]:
texts = []
golden_labels = []

for _, row in decicontas_json.iterrows():
    texto = row['data']['text']
    anotacoes = json.dumps([x['value'] for x in row['annotations'][0]['result']])

    texts.append(texto)
    golden_labels.append(anotacoes)

In [ ]:
def extract_triples(entity_list: List[dict]) -> List[Tuple[int, int, str]]:
    triples = []
    for e in entity_list:
        label = e["labels"]
        if isinstance(label, list):
            label = label[0]
        triples.append((e["start"], e["end"], label))
    return triples

In [54]:
extract_triples(ast.literal_eval(golden_labels[112]))

[]

In [71]:
results_log = []  # Armazena dados por instância
y_true, y_pred = [], []

for i, (text, gold_str) in enumerate(zip(texts, golden_labels)):
    if i % 10 == 0 and i > 0:
        print(f"Processando instância {i} de {len(texts)}")
    try:
        gold = ast.literal_eval(gold_str)
        gold_triples = extract_triples(gold)
        gold_str_repr = str(gold_triples)

        pred = run_chain(text)
        pred_triples = extract_triples(pred)
        pred_str_repr = str(pred_triples)

        # <COMPARE_RESULTS>
        match = pred_str_repr == gold_str_repr
        y_true.append(1)
        y_pred.append(1 if match else 0)

        # <SAVE_RESULTS>
        results_log.append({
            "index": i,
            "text": text,
            "gold": gold_str_repr,
            "pred": pred_str_repr,
            "match": match
        })

    except Exception as e:
        print(f"[Erro na instância {i}] {e}")
        results_log.append({
            "index": i,
            "text": text,
            "gold": gold_str,
            "pred": None,
            "match": False,
            "error": str(e)
        })
        y_true.append(1)
        y_pred.append(0)
        continue


Processando instância 10 de 1337
Processando instância 20 de 1337
Processando instância 30 de 1337
Processando instância 40 de 1337
Processando instância 50 de 1337
Processando instância 60 de 1337
Processando instância 70 de 1337
Processando instância 80 de 1337
Processando instância 90 de 1337
Processando instância 100 de 1337
Processando instância 110 de 1337
Processando instância 120 de 1337
Processando instância 130 de 1337
Processando instância 140 de 1337
Processando instância 150 de 1337
Processando instância 160 de 1337
Processando instância 170 de 1337
Processando instância 180 de 1337
Processando instância 190 de 1337
Processando instância 200 de 1337
Processando instância 210 de 1337
Processando instância 220 de 1337
Processando instância 230 de 1337
Processando instância 240 de 1337
Processando instância 250 de 1337
Processando instância 260 de 1337
Processando instância 270 de 1337
Processando instância 280 de 1337
Processando instância 290 de 1337
Processando instância 3

In [72]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision:", precision_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("F1:", f1_score(y_true, y_pred))


Precision: 1.0
Recall: 0.8511593118922962
F1: 0.9195959595959596


In [73]:
df_results_gpt_4o = pd.DataFrame(results_log)

In [74]:
df_results_gpt_4o

,index,text,gold,pred,match
0,0,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True
1,1,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True
2,2,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True
3,3,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True
4,4,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True
...,...,...,...,...,...
1332,1332,"Vistos, relatados e discutidos estes autos, em...",[],[],True
1333,1333,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True
1334,1334,"Vistos, relatados e discutidos estes autos, co...",[],[],True
1335,1335,DECIDEM os Conselheiros do Tribunal de Contas ...,[],[],True


In [76]:
df_results_gpt_4o[df_results_gpt_4o['match'] == False]

,index,text,gold,pred,match
176,176,"Vistos, relatados e discutidos estes autos,aca...","[(266, 731, 'MULTA_FIXA'), (921, 1277, 'OBRIGA...","[(59, 421, 'MULTA_FIXA'), (503, 849, 'OBRIGACA...",False
177,177,"Vistos, relatados e discutidos estes autos, ac...","[(265, 729, 'MULTA_FIXA'), (916, 1179, 'OBRIGA...","[(116, 361, 'MULTA_FIXA'), (494, 745, 'OBRIGAC...",False
186,186,"Vistos, relatados e discutidos estes autos, ac...","[(311, 755, 'MULTA_FIXA'), (946, 1307, 'OBRIGA...","[(95, 470, 'MULTA_FIXA'), (522, 849, 'OBRIGACA...",False
188,188,"Vistos, relatados e discutidos estes autos, co...","[(362, 896, 'MULTA_FIXA')]","[(255, 645, 'MULTA_FIXA')]",False
189,189,"Vistos, relatados e discutidos estes autos, co...","[(620, 895, 'MULTA_FIXA'), (901, 1190, 'MULTA_...","[(401, 566, 'MULTA_FIXA'), (571, 751, 'MULTA_F...",False
...,...,...,...,...,...
1280,1280,"Vistos, relatados e discutidos estes autos, AC...","[(619, 928, 'RECOMENDACAO')]",[],False
1282,1282,"Vistos, relatados e discutidos estes autos, AC...",[],"[(414, 710, 'RECOMENDACAO')]",False
1284,1284,"Vistos, relatados e discutidos estes autos, co...","[(370, 778, 'MULTA_FIXA')]","[(241, 485, 'MULTA_FIXA')]",False
1311,1311,"Vistos, relatados e discutidos estes autos, em...","[(660, 969, 'RECOMENDACAO')]",[],False


In [ ]:
df_results_gpt_4o.to_csv("dataset/results/decicontas_gpt_4o.csv", index=False)

# Extracting pydantic entities

In [18]:
from tools.prompt import generate_few_shot_ner_prompts
from langchain_core.runnables import RunnableLambda, RunnableMap

prepare_prompt_chain = RunnableLambda(lambda x: generate_few_shot_ner_prompts(x["text"]))
ner_chain = prepare_prompt_chain | llm_azure_gpt_4o



In [19]:
exemplo_erro_1 = decicontas_json.loc[176]['data']['text']
exemplo_erro_2 = decicontas_json.loc[177]['data']['text']
exemplo_erro_3 = decicontas_json.loc[1280]['data']['text']
exemplo_erro_4 = decicontas_json.loc[1336]['data']['text']

In [ ]:
result = ner_chain.invoke({"text": exemplo_erro_1})
pprint.pprint(exemplo_erro_1)
pprint.pprint(result.content)

('Vistos, relatados e discutidos estes autos,acatando o entendimento do '
 'Ministério Público Especial, com fulcro nos fundamentos jurídicos dantes '
 'explanados, ACORDAM os Conselheiros, nos termos do voto proferido pelo '
 'Conselheiro Relator, julgar: \n'
 '\n'
 'a)\tpela APLICAÇÃO DE MULTA no valor de R$1.000,00 (mil reais) para o então '
 'gestor responsável, à época dos fatos, pelo Instituto de Previdência dos '
 'Servidores do Estado do Rio Grande do Norte - IPERN,  senhor Nereu Batista '
 'Linhares, nos termos do artigo 107, inciso II, alínea “f”, da Lei '
 'Complementar Estadual nº 464/2012 c/c o artigo 323, inciso II, alínea `f`, '
 'do Novel Regimento Interno desta Casa, em virtude do descumprimento de '
 'determinação do Tribunal (Decisão nº 1255/2020-TC).\n'
 ' \n'
 'b)\tpela INTIMAÇÃO da referida autoridade competente nominada, a fim de que '
 'tome conhecimento desta decisão e, se for o caso, apresente recurso no prazo '
 'regimental. \n'
 '\n'
 'c)\tpela RENOVAÇÃO DA 

In [22]:
result = ner_chain.invoke({"text": exemplo_erro_2})
pprint.pprint(exemplo_erro_2)
pprint.pprint(result.content)

('Vistos, relatados e discutidos estes autos, acatando o entendimento do '
 'Ministério Público Especial, com fulcro nos fundamentos jurídicos dantes '
 'explanados, ACORDAM os Conselheiros, nos termos do voto proferido pelo '
 'Conselheiro Relator, julgar:\n'
 'a)\tpela APLICAÇÃO DE MULTA no valor de R$1.000,00 (mil reais) para o então '
 'gestor responsável, à época dos fatos, pelo Instituto de Previdência dos '
 'Servidores do Estado do Rio Grande do Norte - IPERN,  senhor Nereu Batista '
 'Linhares, nos termos do artigo 107, inciso II, alínea “f”, da Lei '
 'Complementar Estadual nº 464/2012 c/c o artigo 323, inciso II, alínea `f`, '
 'do Novel Regimento Interno desta Casa, em virtude do descumprimento de '
 'determinação do Tribunal (Decisão nº 917/2020-TC).\n'
 'b)\tpela INTIMAÇÃO da referida autoridade competente nominada, a fim de que '
 'tome conhecimento desta decisão e, se for o caso, apresente recurso no prazo '
 'regimental. \n'
 'c)\tpela RENOVAÇÃO DA DETERMINAÇÃO constan

In [10]:
result = ner_chain.invoke({"text": exemplo_erro_3})
pprint.pprint(exemplo_erro_3)
pprint.pprint(result)

('Vistos, relatados e discutidos estes autos, ACORDAM os(as) Conselheiros(as), '
 'nos termos do voto proferido pelo Conselheiro Relator, em consonância com o '
 'Corpo Técnico da DAM e com o parecer do Ministério Público junto a esta '
 'Corte, julgar no sentido de conhecer e prover o Pedido de Reconsideração '
 '006990/2019-TC – (evento 41) interposto por Kerginaldo Medeiros de Araújo, '
 'para reformar, integralmente, o Acórdão nº 257/2019-TC-2ª Câmara (evento '
 '30), com vistas a:\n'
 '1) Declarar a prescrição da pretensão punitiva em favor do recorrente;\n'
 '2) Determinar o arquivamento do processo após o trânsito em julgado do '
 'Acórdão;\n'
 '3) Recomendar à Secretaria de Controle Externo – SECEX deste Tribunal de '
 'Contas a adoção de procedimentos com vistas a impedir que processos fiquem '
 'paralisados por mais de três anos nas Unidades Técnicas a ela vinculadas, '
 'para que se evite, assim, a ocorrência de prescrição intercorrente, como no '
 'presente caso;\n'
 '4) Sa

In [11]:
result = ner_chain.invoke({"text": exemplo_erro_4})
pprint.pprint(exemplo_erro_4)
pprint.pprint(result)

OutputParserException: Invalid json output: No texto fornecido, as informações extraídas pertencem às categorias de registro de decisão e recomendação. Segue a análise:

**Registro Decisão**  
1. **Categoria:** Decisão sobre ato de admissão  
   **Descrição:** Pelo REGISTRO TÁCITO do ato de admissão em apreço.

2.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [10]:
result

AIMessage(content='```json\n{\n  "multas_fixas": [],\n  "multas_percentuais": [],\n  "obrigacoes_multa": [],\n  "ressarcimentos": [],\n  "obrigacoes": [],\n  "recomendacoes": []\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 8770, 'total_tokens': 8823, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BmTcTtbANZk6jrQLO6cOfhxHSmArE', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severit

In [20]:
decicontas_json['result'] = decicontas_json['annotations'].apply(lambda x: x[0]['result'])

/tmp/ipykernel_108595/2483412639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decicontas_json['result'] = decicontas_json['annotations'].apply(lambda x: x[0]['result'])


In [23]:
decicontas_json.loc[176]['data']['text']

'Vistos, relatados e discutidos estes autos,acatando o entendimento do Ministério Público Especial, com fulcro nos fundamentos jurídicos dantes explanados, ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro Relator, julgar: \n\na)\tpela APLICAÇÃO DE MULTA no valor de R$1.000,00 (mil reais) para o então gestor responsável, à época dos fatos, pelo Instituto de Previdência dos Servidores do Estado do Rio Grande do Norte - IPERN,  senhor Nereu Batista Linhares, nos termos do artigo 107, inciso II, alínea “f”, da Lei Complementar Estadual nº 464/2012 c/c o artigo 323, inciso II, alínea `f`, do Novel Regimento Interno desta Casa, em virtude do descumprimento de determinação do Tribunal (Decisão nº 1255/2020-TC).\n \nb)\tpela INTIMAÇÃO da referida autoridade competente nominada, a fim de que tome conhecimento desta decisão e, se for o caso, apresente recurso no prazo regimental. \n\nc)\tpela RENOVAÇÃO DA DETERMINAÇÃO constante na decisão retro, estipulando o prazo de 30 (t